In [1]:
# Notebook for estimating development from Rt and cases
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (14,10)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.linewidth'] = 3
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
locale.setlocale(locale.LC_ALL,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

import os
# import csv
import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
#Settings for plotting
fs_label = 14

parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label
             }

plt.rcParams.update(parameters)

#locale.setlocale(locale.LC_TIME, "Danish") 

In [3]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

In [4]:
dfCaseDiff = pd.read_csv('DailyCasesAveraged.csv')

In [18]:
curDates = pd.to_datetime(dfCaseDiff.Dato).values
plotDates = np.arange(curDates[0],curDates[-1]+np.timedelta64(1,'D'),np.timedelta64(1,'D'))

allCols = dfCaseDiff.columns[2:]

In [20]:
fig,ax1 = plt.subplots(1,1)

curCol = '20-29'
ax1.plot(plotDates,dfCaseDiff[curCol],'*-')


plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:


# fig,ax1 = plt.subplots(1,1)
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,figsize=(16,10), gridspec_kw={'height_ratios': [3, 1]})

showMax = False

# tStart = np.datetime64('2021-04-13')
tStart = np.datetime64('2021-04-20')

tEndWeeks = 15
tDelta = np.timedelta64(7,'D')

tSpan = np.arange(0,tEndWeeks) # Time span, in number of weeks since tStart
tSpanPlot = np.arange(tStart,tStart+np.timedelta64(7*tEndWeeks,'D'),tDelta) # Time span, as datetime64

xLims = [tStart-tDelta*3,tSpanPlot[-1]]

iniRow = dfCaseDiff.iloc[plotDates == tStart]
iniRow


curAge = '30-39'
curData = dfCaseDiff[curAge].values
iniCount = iniRow[curAge].values[0]

ax1.set_title(curAge+' årige')

genTime = 4.7 / 7 # Generation time, in weeks

allRTs = np.arange(1.0,1.8,0.1)
# allRTs = np.arange(1.0,1.5,0.1)
# allRTs = np.arange(1.0,1.3,0.1)

lastDateStr = pd.to_datetime(tSpanPlot[-1]).strftime('%d. %b')

for curRT in allRTs:
    
    modelCount = iniCount * (curRT**(genTime*tSpan))
    modelSum = modelCount.sum()
    ax1.plot(tSpanPlot,modelCount,'.-',label=f'Fremskrivning med RT: {curRT:.1f}'.replace('.',','))
    ax2.plot(tSpanPlot,np.cumsum(modelCount),'.-',label=f'RT: {curRT:.1f}'.replace('.',',')+f'\nTotal: {modelSum:,.0f} indtil {lastDateStr}'.replace(',','.')) 
    # ax1.plot(tSpanPlot,modelCount,'.-',label=f'Fremskrivning med RT: {curRT:.1f}'.replace('.',',')+f' \nTotal: {modelSum:,.0f} indtil {lastDateStr}'.replace(',','.')) 


ax1.plot(plotDates,curData,'k*:',linewidth=0.5,label=f'Data for {curAge} årige')

if showMax:
    dataMax = max(curData)
    ax1.plot(plotDates,dataMax * np.ones(curData.shape),color='grey',linewidth=1,label='Maksimalt antal smittede, anden bølge')
    ax1.set_ylim(bottom = 0,top=dataMax*1.1)
else:
    ax1.set_ylim(bottom = 0,top=iniCount*2.5)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

ax1.set_xlim(xLims)
# ax1.set_xlim([tSpanPlot[0],tSpanPlot[-1]])
# ax1.set_xlim([tSpanPlot[0],np.datetime64('2021-06-01')])

ax1.set_ylabel('Ugentlige smittetilfælde')

ax2.set_yscale('log')
ax2.set_ylabel('Kummulerede smittetilfælde \n[logaritmisk akse]')
# ax2.grid(axis='y')  

ax1.grid()
ax2.grid()

# ax1.legend()
ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
# Calculation of size of age-group
dfAges = pd.read_excel('DKfolketal2021clean.xlsx')
DK_age = []
DK_age.append(dfAges.iloc[0:10].Antal.sum())
DK_age.append(dfAges.iloc[10:20].Antal.sum())
DK_age.append(dfAges.iloc[20:30].Antal.sum())
DK_age.append(dfAges.iloc[30:40].Antal.sum())
DK_age.append(dfAges.iloc[40:50].Antal.sum())
DK_age.append(dfAges.iloc[50:60].Antal.sum())
DK_age.append(dfAges.iloc[60:70].Antal.sum())
DK_age.append(dfAges.iloc[70:80].Antal.sum())
DK_age.append(dfAges.iloc[80:90].Antal.sum())
DK_age.append(dfAges.iloc[90:].Antal.sum())
DK_age.append(dfAges.Antal.sum())
DK_age = np.array(DK_age)
DK_age

array([ 610210,  680040,  778740,  696679,  748824,  801166,  667583,
        574697,  236648,   45458, 5840045], dtype=int64)

In [27]:
# Vaccinationskalender
vaccKal = pd.DataFrame()

neverDate = np.datetime64('2021-09-30')
vaccKal['0-9'] = [neverDate,neverDate,neverDate]
vaccKal['10-19'] = [np.datetime64('2021-05-24'),np.datetime64('2021-06-14'),np.datetime64('2021-07-19')]
vaccKal['20-29'] = [np.datetime64('2021-06-14'),np.datetime64('2021-07-19'),np.datetime64('2021-08-31')]
vaccKal['30-39'] = [np.datetime64('2021-06-21'),np.datetime64('2021-08-01'),np.datetime64('2021-08-31')]
vaccKal['40-49'] = [np.datetime64('2021-05-24'),np.datetime64('2021-06-28'),np.datetime64('2021-08-02')]
vaccKal['50-59'] = [np.datetime64('2021-05-03'),np.datetime64('2021-05-31'),np.datetime64('2021-07-05')]
vaccKal['60-69'] = [np.datetime64('2021-04-05'),np.datetime64('2021-05-24'),np.datetime64('2021-06-28')]
vaccKal['70-79'] = [np.datetime64('2021-03-29'),np.datetime64('2021-04-26'),np.datetime64('2021-05-31')]
vaccKal['80-89'] = [np.datetime64('2021-02-01'),np.datetime64('2021-03-29'),np.datetime64('2021-04-26')]
vaccKal['90+'] =   [np.datetime64('2021-02-01'),np.datetime64('2021-02-22'),np.datetime64('2021-03-15')]
# vaccKal['I alt'] = [np.datetime64('2021-07-19'),np.datetime64('2021-08-02'),np.datetime64('2021-08-31')]
vaccKal['I alt'] = [neverDate,neverDate,neverDate]

vaccKal

,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,I alt
0,2021-09-30,2021-05-24,2021-06-14,2021-06-21,2021-05-24,2021-05-03,2021-04-05,2021-03-29,2021-02-01,2021-02-01,2021-09-30
1,2021-09-30,2021-06-14,2021-07-19,2021-08-01,2021-06-28,2021-05-31,2021-05-24,2021-04-26,2021-03-29,2021-02-22,2021-09-30
2,2021-09-30,2021-07-19,2021-08-31,2021-08-31,2021-08-02,2021-07-05,2021-06-28,2021-05-31,2021-04-26,2021-03-15,2021-09-30


In [50]:
# iniDateToUseIndex = np.where(plotDates == tStart)[0][0]

# daysToSumUp = 7
# curIniAverage = dfCaseDiff[curAge].iloc[iniDateToUseIndex-daysToSumUp:iniDateToUseIndex].values.sum()/daysToSumUp
# curIniAverage

In [58]:
# Forskellige Rt for aldersgruppe, hvor mange ugentligt smittede har vi før gruppen er vaccineret

fig,ax1 = plt.subplots(1,1,figsize=(18,12))
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True, gridspec_kw={'height_ratios': [4, 1]})

showMax = False

# tStart = np.datetime64('2021-04-12')
# tStart = np.datetime64('2021-04-13')
# tStart = np.datetime64('2021-04-27')
tStart = plotDates[-1]


tEndWeeks = 22
tDelta = np.timedelta64(7,'D')

tSpan = np.arange(0,tEndWeeks) # Time span, in number of weeks since tStart
tSpanPlot = np.arange(tStart,tStart+np.timedelta64(7*tEndWeeks,'D'),tDelta) # Time span, as datetime64
tSpanPlot -= np.timedelta64(3,'D')

xLims = [tStart-tDelta*4,tSpanPlot[-1]]

iniRow = dfCaseDiff.iloc[plotDates == tStart]

# curAge = '20-29'
curAge = '30-39'
# curAge = '40-49'
# curAge = '50-59'
# vaccIni = np.datetime64('2021-06-21')
# vaccOne = np.datetime64('2021-08-01')
# vaccTwo = np.datetime64('2021-08-31')
vaccIni = vaccKal[curAge][0]
vaccOne = vaccKal[curAge][1]
vaccTwo = vaccKal[curAge][2]

curData = dfCaseDiff[curAge].values
# iniCount = iniRow[curAge].values[0]

iniDateToUseIndex = np.where(plotDates == tStart)[0][0]

daysToSumUp = 7
curIniAverage = dfCaseDiff[curAge].iloc[iniDateToUseIndex-daysToSumUp+1:iniDateToUseIndex+1].values.sum()/daysToSumUp
iniCount = curIniAverage


curMax = 400/7
ax1.fill_between([vaccIni,vaccOne],[curMax,curMax],color='aquamarine',label='Vaccination start')
ax1.fill_between([vaccOne,vaccTwo],[curMax,curMax],color='palegreen',label='Første dosis')
ax1.fill_between([vaccTwo,tSpanPlot[-1]],[curMax,curMax],color='springgreen',label='Færdigvaccineret')

ax1.set_title(curAge+' årige')

genTime = 4.7 / 7 # Generation time, in weeks

allRTs = np.arange(1.0,1.8,0.1)
# allRTs = np.arange(1.0,1.3,0.1)

lastDateStr = pd.to_datetime(tSpanPlot[-1]).strftime('%d. %b')

cmap = plt.cm.get_cmap('plasma',len(allRTs))
# for curRT in allRTs:
for k in range(0,len(allRTs)):
    curRT = allRTs[k]
    
    modelCount = iniCount * (curRT**(genTime*tSpan))
    modelSum = modelCount.sum()
    # ax1.plot(tSpanPlot,modelCount,'.-',label=f'RT: {curRT:.1f}'.replace('.',','),color=cmap(k))
    # ax1.plot(tSpanPlot,modelCount,'.-',label=f'Fremskrivning med RT: {curRT:.1f}'.replace('.',','),color=cmap(k))
    
    ax1.plot(tSpanPlot,modelCount/7,'.-',label=f'RT: {curRT:.1f}'.replace('.',','),color=cmap(k))


# ax1.plot(plotDates,curData,'k*:',linewidth=0.5,label=f'Data (SSI)')
# ax1.plot(plotDates,curData,'k*:',linewidth=0.5,label=f'Data for {curAge} årige')

ax1.plot(plotDates,curData/7,'k*:',linewidth=0.5,label=f'Data (SSI)')
meanWidth = 7
ax1.plot(rnTime(plotDates,meanWidth),rnMean(curData/7,meanWidth),'k',label=f'Data (SSI)')

# ax1.plot([plotDates[0],tSpanPlot[-1]],[375,375],'r',linewidth=5,label='Nedlukningsgrænse')
ax1.plot([plotDates[0],tSpanPlot[-1]],[375/7,375/7],'r',linewidth=5,label='Nedlukningsgrænse')


if showMax:
    dataMax = max(curData)
    ax1.plot(plotDates,dataMax * np.ones(curData.shape),color='grey',linewidth=1,label='Maksimalt antal smittede\nAnden bølge')
    ax1.set_ylim(bottom = 0,top=dataMax*1.1)
else:
    # ax1.set_ylim(bottom = 0,top=iniCount*8/7)
    ax1.set_ylim(bottom = 0,top=curMax)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))

ax1.set_xlim(xLims)

# ax1.set_ylabel('Ugentlige smittetilfælde')
ax1.set_ylabel('Incidens per 100000')
ax1.set_xlabel('Dato')

ax1.grid(color='black')

# ax1.legend()
ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# Forskellige Rt for aldersgruppe, hvor mange ugentligt smittede har vi før gruppen er vaccineret

# fig,ax1 = plt.subplots(1,1,figsize=(20,10))
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True, gridspec_kw={'height_ratios': [4, 1]})

showMax = False

# tStart = np.datetime64('2021-04-13')
# tStart = np.datetime64('2021-04-27')
tStart = plotDates[-1]

tEndWeeks = 18
tDelta = np.timedelta64(7,'D')

tSpan = np.arange(0,tEndWeeks) # Time span, in number of weeks since tStart
tSpanPlot = np.arange(tStart,tStart+np.timedelta64(7*tEndWeeks,'D'),tDelta) # Time span, as datetime64

xLims = [tStart-tDelta*4,tSpanPlot[-1]]

iniRow = dfCaseDiff.iloc[plotDates == tStart]
iniRow

curCols = dfCaseDiff.columns
for i in range(0,len(curCols)):
    
    fig,ax1 = plt.subplots(1,1,figsize=(20,10))
    curAge = curCols[i]
    curPopSize = DK_age[i]

    # # curAge = '20-29'
    # curAge = '30-39'
    # # curAge = '40-49'
    # # curAge = '50-59'

    # curPopSize = DK_age[3]

    vaccIni = vaccKal[curAge][0]
    vaccOne = vaccKal[curAge][1]
    vaccTwo = vaccKal[curAge][2]


    curData = dfCaseDiff[curAge].values
    iniCount = iniRow[curAge].values[0]



    # ax1.bar(vaccIni,max(curData),color='springgreen',label='Vaccination påbegyndes')
    # ax1.bar(vaccOne,max(curData),color='limegreen',label='Første dosis afsluttet')
    # ax1.bar(vaccTwo,max(curData),color='darkgreen',label='Forventet færdigvaccineret')

    # ax1.fill_between([vaccIni,vaccOne],[max(curData),max(curData)],color='springgreen',label='Vaccination start')
    # ax1.fill_between([vaccOne,vaccTwo],[max(curData),max(curData)],color='limegreen',label='Første dosis')
    # ax1.fill_between([vaccTwo,tSpanPlot[-1]],[max(curData),max(curData)],color='forestgreen',label='Færdigvaccineret')

    curMax = 400
    ax1.fill_between([vaccIni,vaccOne],[curMax,curMax],color='aquamarine',label='Vaccination start')
    ax1.fill_between([vaccOne,vaccTwo],[curMax,curMax],color='palegreen',label='Første dosis')
    ax1.fill_between([vaccTwo,tSpanPlot[-1]],[curMax,curMax],color='springgreen',label='Færdigvaccineret')

    ax1.set_title(curAge+' årige')
    if (curAge == 'I alt'):
        ax1.set_title(curAge)


    genTime = 4.7 / 7 # Generation time, in weeks

    allRTs = np.arange(1.0,1.8,0.1)
    # allRTs = np.arange(1.0,1.3,0.1)

    lastDateStr = pd.to_datetime(tSpanPlot[-1]).strftime('%d. %b')

    cmap = plt.cm.get_cmap('plasma',len(allRTs))
    # for curRT in allRTs:
    for k in range(0,len(allRTs)):
        curRT = allRTs[k]
        
        modelCount = iniCount * (curRT**(genTime*tSpan))
        modelSum = modelCount.sum()
        # ax1.plot(tSpanPlot,modelCount,'.-',label=f'RT: {curRT:.1f}'.replace('.',','),color=cmap(k))
        # ax1.plot(tSpanPlot,modelCount,'.-',label=f'Fremskrivning med RT: {curRT:.1f}'.replace('.',','),color=cmap(k))
        
        # ax1.plot(tSpanPlot,modelCount/7,'.-',label=f'RT: {curRT:.1f}'.replace('.',','),color=cmap(k))
        ax1.plot(tSpanPlot,100000 * modelCount/curPopSize,'.-',linewidth=1.5,markersize=6,label=f'RT: {curRT:.1f}'.replace('.',','),color=cmap(k))


    # ax1.plot(plotDates,curData,'k*:',linewidth=0.5,label=f'Data (SSI)')
    # ax1.plot(plotDates,curData,'k*:',linewidth=0.5,label=f'Data for {curAge} årige')

    # ax1.plot(plotDates,curData/7,'k*:',linewidth=0.5,label=f'Data (SSI)')
    ax1.plot(plotDates,100000 * curData/curPopSize,'k*:',linewidth=0.5,label=f'Data (SSI)')

    ax1.plot([plotDates[0],tSpanPlot[-1]],[375,375],'r',linewidth=5,label='Nedlukningsgrænse')


    if showMax:
        dataMax = max(curData)
        ax1.plot(plotDates,dataMax * np.ones(curData.shape),color='grey',linewidth=1,label='Maksimalt antal smittede\nAnden bølge')
        ax1.set_ylim(bottom = 0,top=dataMax*1.1)
    else:
        # ax1.set_ylim(bottom = 0,top=iniCount*8/7)
        ax1.set_ylim(bottom = 0,top=curMax)

    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))

    ax1.set_xlim(xLims)

    # ax1.set_ylabel('Ugentlige smittetilfælde')
    ax1.set_ylabel('Incidens per 100.000 borgere')
    ax1.set_xlabel('Dato')

    # ax1.grid(color='black')
    ax1.grid()

    # ax1.legend()
    ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

    plt.tight_layout()

    # if saveFigures:
    #     plt.savefig('figs/RT_fremskrivning_Per100000_Dagligt_'+curAge)
# plt.close('all')